In [14]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.linear_model import LinearRegression 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder

dataset = pd.read_csv('Salary_Data.csv')
# load dataset
dataset

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52.0,Male,Master's,Director,20.0,200000.0
...,...,...,...,...,...,...
6699,49.0,Female,PhD,Director of Marketing,20.0,200000.0
6700,32.0,Male,High School,Sales Associate,3.0,50000.0
6701,30.0,Female,Bachelor's Degree,Financial Manager,4.0,55000.0
6702,46.0,Male,Master's Degree,Marketing Manager,14.0,140000.0


In [15]:
dataset.shape

(6704, 6)

In [16]:
dataset.isnull().sum()

Age                    2
Gender                 2
Education Level        3
Job Title              2
Years of Experience    3
Salary                 5
dtype: int64

In [20]:
# Creating a LabelEncoder object
lb_make = LabelEncoder()

# List of categorical columns to be transformed
categorical_cols = ["Gender", "Education Level", "Job Title"]

# Applying LabelEncoder on train data
for col in categorical_cols:
    dataset[col] = lb_make.fit_transform(dataset[col])

dataset

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,1,0,177,5.0,90000.0
1,28.0,0,3,18,3.0,65000.0
2,45.0,1,5,145,15.0,150000.0
3,36.0,0,0,116,7.0,60000.0
4,52.0,1,3,26,20.0,200000.0
...,...,...,...,...,...,...
6699,49.0,0,5,34,20.0,200000.0
6700,32.0,1,2,116,3.0,50000.0
6701,30.0,0,1,42,4.0,55000.0
6702,46.0,1,4,97,14.0,140000.0


In [23]:
#split dataset in features and target variable
features = ['Age', 'Gender', 'Education Level', 'Years of Experience', 'Job Title']
X = dataset.loc[:, features]
y = dataset.loc[:, 'Salary']

In [25]:
regressor = LinearRegression()
regressor.fit(X, y)

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values